In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a world-class international chef.. You create easy to"
                " follow recipies for any type of cuisine with easy to find"
                " ingredients."
            ),
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a vegetarian chef specialized on making traditional"
                " recipes vegetarian. You find alternative ingredients and"
                " explain their preparation. You don't radically modify the"
                " recipe. If there is no alternative for a food just say you"
                " don't know how to recipe it."
            ),
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "Korean"})

Great choice! Korean cuisine is known for its bold flavors and unique ingredients. Let's start with a classic and delicious recipe for Bibimbap, a popular Korean mixed rice dish. Here's how you can make it at home:

Ingredients:
- 2 cups cooked short-grain rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound beef (you can use ribeye or sirloin), thinly sliced
- 4 eggs
- Sesame oil
- Soy sauce
- Gochujang (Korean red chili paste)
- Sesame seeds
- Salt
- Vegetable oil

Instructions:
1. Marinate the beef: In a bowl, mix the beef slices with 2 tablespoons of soy sauce, 1 tablespoon of sesame oil, and a pinch of black pepper. Let it marinate for at least 30 minutes.

2. Prepare the vegetables:
- Carrot: Heat a pan with vegetable oil and stir-fry the julienned carrot until slightly softened. Season with a pinch of salt and set aside.
- Zucchini: Repeat the same process for the julienned zucchini.
- Spinach: Blanch 

AIMessageChunk(content="For a vegetarian version of Bibimbap, you can make a few simple swaps to replace the beef in the traditional recipe. Here's how you can adapt the recipe:\n\nAlternative Ingredients:\n- Instead of beef, you can use tofu as a protein-rich substitute. Extra-firm tofu works well for this dish.\n- You can also consider using tempeh as another plant-based alternative to beef. Tempeh has a nutty flavor and a firm texture that can complement the other ingredients in Bibimbap.\n\nPreparation:\n1. Marinate the tofu or tempeh: Cut the tofu or tempeh into thin slices or cubes. Marinate them in a mixture of soy sauce, sesame oil, and a pinch of black pepper for at least 30 minutes to allow the flavors to infuse.\n\n2. Cook the tofu or tempeh: In a pan, heat some vegetable oil and cook the marinated tofu or tempeh until browned and slightly crispy on the outside. Set aside once cooked.\n\n3. Follow the rest of the recipe as instructed, preparing the vegetables and eggs, and a